In [32]:
from google.cloud.aiplatform.prediction import LocalModel 
from src.custom_sam_predictor import CustomSamPredictor 
import os
import logging

In [33]:
logging.basicConfig(level=logging.INFO)

In [34]:
USER_SRC_DIR = "src"
REGION = "us-west1"
REPOSITORY = "sam-container"  # @param {type:"string"}
PROJECT_ID = "ml-ops-segment-anything"
IMAGE = "sam-cpr-container"  # @param {type:"string"}
LOCAL_MODEL_ARTIFACTS_DIR = "model_artifacts"

In [41]:
#!docker system prune -f

In [15]:
#!docker <container/image/builder> prune --all
#!docker system df

In [35]:
local_model = LocalModel.build_cpr_model(
    USER_SRC_DIR,
    f"{REGION}-docker.pkg.dev/{PROJECT_ID}/{REPOSITORY}/{IMAGE}",
    predictor=CustomSamPredictor,
    requirements_path=os.path.join(USER_SRC_DIR, "requirements.txt"),
    base_image= "pytorch/pytorch:1.13.1-cuda11.6-cudnn8-runtime"
)

INFO:google.cloud.aiplatform.docker_utils.build:Running command: docker build -t us-west1-docker.pkg.dev/ml-ops-segment-anything/sam-container/sam-cpr-container --rm -f- src
/opt/conda/lib/python3.10/subprocess.py:955: RuntimeWarning: line buffering (buffering=1) isn't supported in binary mode, the default buffer size will be used
  self.stdin = io.open(p2cwrite, 'wb', bufsize)
/opt/conda/lib/python3.10/subprocess.py:961: RuntimeWarning: line buffering (buffering=1) isn't supported in binary mode, the default buffer size will be used
  self.stdout = io.open(c2pread, 'rb', bufsize)
INFO:google.cloud.aiplatform.docker_utils.local_util:Sending build context to Docker daemon  375.4MB
INFO:google.cloud.aiplatform.docker_utils.local_util:

INFO:google.cloud.aiplatform.docker_utils.local_util:Step 1/14 : FROM pytorch/pytorch:1.13.1-cuda11.6-cudnn8-runtime

INFO:google.cloud.aiplatform.docker_utils.local_util: ---> 71eb2d092138

INFO:google.cloud.aiplatform.docker_utils.local_util:Step 2/14 : 

In [36]:
local_model.get_serving_container_spec()

image_uri: "us-west1-docker.pkg.dev/ml-ops-segment-anything/sam-container/sam-cpr-container"
predict_route: "/predict"
health_route: "/health"

In [37]:
INPUT_FILE_WITH_PROMPTS = "/home/jupyter/input_with_prompts.json"
INPUT_FILE_WITHOUT_PROMPTS = "/home/jupyter/input_without_prompts.json"

In [38]:
with local_model.deploy_to_local_endpoint(
    artifact_uri=f"{LOCAL_MODEL_ARTIFACTS_DIR}",
    gpu_count=1
    
) as local_endpoint:
    print("predicting")
    predict_response = local_endpoint.predict(
        request_file=INPUT_FILE_WITHOUT_PROMPTS,
        headers={"Content-Type": "application/json"},
    )

    health_check_response = local_endpoint.run_health_check()

INFO:google.cloud.aiplatform.prediction.local_endpoint:Got the project id from the global config: ml-ops-segment-anything.


predicting


In [40]:
#print(predict_response.json())

In [44]:
#!gcloud auth configure-docker us-west1-docker.pkg.dev --quiet
     

In [45]:
#!gcloud services enable artifactregistry.googleapis.com

In [41]:
local_model.push_image()

/opt/conda/lib/python3.10/subprocess.py:955: RuntimeWarning: line buffering (buffering=1) isn't supported in binary mode, the default buffer size will be used
  self.stdin = io.open(p2cwrite, 'wb', bufsize)
/opt/conda/lib/python3.10/subprocess.py:961: RuntimeWarning: line buffering (buffering=1) isn't supported in binary mode, the default buffer size will be used
  self.stdout = io.open(c2pread, 'rb', bufsize)
INFO:google.cloud.aiplatform.docker_utils.local_util:Using default tag: latest

INFO:google.cloud.aiplatform.docker_utils.local_util:The push refers to repository [us-west1-docker.pkg.dev/ml-ops-segment-anything/sam-container/sam-cpr-container]

INFO:google.cloud.aiplatform.docker_utils.local_util:b19630e152f7: Preparing

INFO:google.cloud.aiplatform.docker_utils.local_util:6ac77ccd5639: Preparing

INFO:google.cloud.aiplatform.docker_utils.local_util:5b9d9543dac2: Preparing

INFO:google.cloud.aiplatform.docker_utils.local_util:57b0b928d4cd: Preparing

INFO:google.cloud.aiplatform

In [44]:
!curl \
-X POST \
-H "Authorization: Bearer $(gcloud auth print-access-token)" \
-H "Content-Type: application/json" \
https://us-west1-aiplatform.googleapis.com/v1/projects/633534855904/locations/us-west1/endpoints/1883850446473265152:predict \
-d "@input_with_prompts.json"

{"detail":"The following exception has occurred: KeyError. Arguments: ('image_cvtColor',)."}